In [10]:
import os
HOME = os.getcwd()

%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))



/content/weights
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-8plb2prn
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-8plb2prn
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
/content/weights
/content/weights/weights
/content/weights/weights/sam_vit_h_4b8939.pth ; exist: True


In [11]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
import cv2
import supervision as sv
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch

In [13]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [14]:
mask_generator = SamAutomaticMaskGenerator(sam)

## Questa sezione serve per non processare immagini di cui si ha già la maschera

In [ ]:
dir_path_masks = "/content/drive/MyDrive/Colab_Notebooks/Machine_Learning/Progetto/Dataset/Masks/"
name_masks = os.listdir(dir_path_masks)
dir_path = "/content/drive/MyDrive/Colab_Notebooks/Machine_Learning/Progetto/Dataset/Images/"
files = os.listdir(dir_path)



name_masks2 = []
for i in range(len(name_masks)):
  name_masks2.append(name_masks[i].replace("_mask.png", "")) 

files2 = []
for i in range(len(files)):
  files2.append(files[i].replace(".jpg", ""))

set1 = set(name_masks2)
set2 = set(files2)
elementi_comuni = set1.intersection(set2)

elementi_comuni_lista = list(elementi_comuni)

for elemento in elementi_comuni_lista:
    elemento = elemento + ".jpg"
    files.remove(elemento)


num_image = len(files)

In [ ]:
num_image

In [ ]:
for i in range(num_image):
  IMAGE_NAME = files[i]
  IMAGE_PATH = dir_path + IMAGE_NAME


  image_bgr = cv2.imread(IMAGE_PATH)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  sam_result = mask_generator.generate(image_rgb)

  new_masks = sorted(sam_result, key=lambda x: x['area'], reverse=True)
  
  if new_masks[0]["bbox"][:2]== [0,0]:  
    new_masks.pop(0)                    

  img_np = (new_masks[0]['segmentation'])
  img_tt = torch.from_numpy(img_np)
  img_tt = img_tt[None, : ]

  img = img_tt.long().type(torch.uint8)

  MASK_NAME = IMAGE_NAME[:-4] + "_mask" + IMAGE_NAME[-4] + "png"
  MASK_PATH = dir_path.replace("Images", "Masks", 1) + MASK_NAME

  tensor = img.permute(1, 2, 0)
  array = tensor.numpy()
  cv2.imwrite(MASK_PATH, array)